## Part c): Adding Lasso for the Franke function

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sys
sys.path.append("../")
import functions as f
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
#Using a standardized dataset 
#np.random.seed(2024)

#x = np.arange(0, 1, 0.05)
#y = np.arange(0, 1, 0.05)
#z = f.FrankeFunction(x, y)
#z = z + np.random.randn(z.shape[0])

np.random.seed(42)
N = 50
x = np.random.rand(N)
y = np.random.randn(N)
z = f.FrankeFunction(x,y) #Using x,y and z when doing regression
z = z + np.random.normal(0, 0.1, z.shape) #the noise was too high, tried sligtly less
#Creating lambda values avoiding "b" in the name as it is a reserved keyword in python
#lamda = np.array([0.1,0.01,0.001,0.0001,0.00001])  # Try values in a small range
lamda = np.logspace(-6, -1, 6)
print(lamda)

degrees = np.arange(1, 6)

In [ ]:

# Empty lists to store scores and parameters
beta_lasso_values = []
mse_lasso_scores = []
r2_lasso_scores = []

# Looping through each lambda
for i in range(np.size(lamda)):
    # Polynomial degrees

    # Appending lists to store scores and parameters
    beta_lasso_values.append([])
    mse_lasso_scores.append([])
    r2_lasso_scores.append([])


    # Looping through each degree
    for degree in degrees:

        # Creating design matrix
        #X = create_design_matrix(x, degree)
        poly_features = PolynomialFeatures(degree=degree)
        X = poly_features.fit_transform(np.column_stack((x, y)))

        # Split the data into training and test data
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=42)

        # Scale and center the data
        scaler = StandardScaler(with_std=True, with_mean=False)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        
        
        # Create and fit the linear regression model
        model = Lasso(alpha = lamda[i], fit_intercept=False, max_iter=100000)
        model.fit(X_train, z_train)
        
        # Make predictions for training and test data
        z_train_pred = model.predict(X_train)
        z_test_pred = model.predict(X_test)
        
        # Getting the coefficients (beta values)
        beta_lasso = model.coef_

        # Compute mean squared error for training and test data
        mse_train = mean_squared_error(z_train, z_train_pred)
        mse_test = mean_squared_error(z_test, z_test_pred)

        # Compute R2 score for training and test data
        r2_train = r2_score(z_train, z_train_pred)
        r2_test = r2_score(z_test, z_test_pred)
        
        # Appending beta values and scores
        beta_lasso_values[i].append(beta_lasso)
        mse_lasso_scores[i].append(mse_test)
        r2_lasso_scores[i].append(r2_test)

#print(beta_lasso_values)
#print(mse_lasso_scores)
#print(r2_lasso_scores)

In [ ]:
# Plotting MSE and R2 scores
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('MSE as a function of Polynomial Degree')

plt.subplot(1, 2, 2)
plt.xlabel('Polynomial Degree')
plt.ylabel('R2')
plt.title('R2 as a function of Polynomial Degree')

for j in range(np.size(lamda)):
    plt.subplot(1, 2, 1)
    plt.plot(degrees, mse_lasso_scores[j], marker='o', label=f'Lambda = {lamda[j]:.5f}')
    plt.subplot(1, 2, 2)
    plt.plot(degrees, r2_lasso_scores[j], marker='o', label=f'Lambda = {lamda[j]:.5f}')

plt.tight_layout()
plt.legend()
plt.show()
plt.savefig(fname="../../results/ridge_error_degree.png")

In [ ]:
# Plotting beta values
plt.figure(figsize=(10, 5))
plt.xlabel('Polynomial Degree')
plt.ylabel('Beta Values')
plt.title('Beta Values as a function of Polynomial Degree')

#colors = ['red', 'blue', 'green', 'orange', 'purple']  # Define colors for each lambda

for i in range(len(beta_lasso_values)):
    for j in range(len(beta_lasso_values[i])):
        for k in range(len(beta_lasso_values[i][j])):
            beta_i = beta_lasso_values[i][j][k]
            plt.plot(degrees[j]*np.ones_like(beta_lasso_values[i][j][k]), beta_lasso_values[i][j][k], marker='o', label= f'Lambda = {lamda[i]:.5f}' if k == 0 else None)  # Use color based on lambda color=colors[i]

plt.tight_layout()
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label=f'Lambda = {lamda[i]:.5f}', markersize=8) for i in range(len(lamda))]) #, markerfacecolor=colors[i
plt.show()


In this cell a "critical discussion of the three methods and a judgement of which model fits the data best" is required.